## Load Metadata - TPCDS
- Prepare metadata and merge into the control table
- Clone notebook for each source schema

In [0]:
-- Create temporary view containing table specific metadata
-- This will be joined with data from information_schema

create or replace temp view table_details as
select
  col1 as table,
  if(col2 = "y", true, false) as load_partitioned,
  split(col3, ',') as cluster_cols
FROM
  (
    VALUES
      ("catalog_returns", "y", "cr_returned_date_sk"),
      ("catalog_sales",   "y", "cs_sold_date_sk"),
      ("inventory",       "y", "inv_item_sk"),
      ("store_returns",   "y", "sr_returned_date_sk"),
      ("store_sales",     "y", "ss_sold_date_sk"),
      ("web_returns",     "y", "wr_returned_date_sk"),
      ("web_sales",       "y", "ws_sold_date_sk")
  );

In [0]:
use catalog identifier(:catalog);
use schema identifier(:schema);

create or replace temporary view control_src as
select
  'lakefed_ingest' as job_name,
  'tpcds' as task_collection,
  'sqlserver' as src_type,
  t.table_catalog as src_catalog,
  t.table_schema as src_schema,
  t.table_name as src_table,
  'lakefed_ingest' as sink_catalog,
  'tpcds_sf1000' as sink_schema,
  t.table_name as sink_table,
  False as enable_iceberg_reads,
  array() as primary_key,
  coalesce(d.cluster_cols, array()) as sink_cluster_cols,
  'full' as load_type,
  coalesce(d.load_partitioned, false) as load_partitioned,
  '*' as select_list,
  null as watermark_col_name,
  null as watermark_col_type,
  null as watermark_col_start_value,
  case when load_partitioned is true then element_at(cluster_cols, 1) else null end as partition_col,
  case when load_partitioned is true then 2048 else null end as partition_size_mb,
  true as task_enabled
from tpcds.information_schema.tables t
left join table_details d
on t.table_name = d.table
where t.table_catalog = 'tpcds'
  and t.table_schema = 'sf_1000_liquid';

select * from control_src

In [0]:
merge with schema evolution into identifier(:table) as t
using control_src as s
on t.src_catalog = s.src_catalog
and t.src_schema = s.src_schema
and t.src_table = s.src_table
when matched
  then update set *
when not matched
  then insert *